# Imports
This code imports several libraries in order to perform some data processing tasks. The libraries used are:

requests: A library that is used to make HTTP requests to web servers and retrieve the response data.

functools: A module that provides several tools for functional programming in Python, including tools for working with higher-order functions.

pathlib: A library that provides an object-oriented interface for working with files and directories on your system.

pickle: A library that provides a way to serialize and deserialize Python objects to and from binary data streams.

os: A library that provides a way to interact with the operating system, such as reading and writing files, checking the current directory, and executing shell commands.

sqlite3: A library that provides a way to interact with SQLite databases, including creating and executing SQL queries.

json: A library that provides a way to encode and decode JSON data.

cv2: A library that provides an interface to the OpenCV computer vision library, allowing you to perform tasks such as image processing, object detection, and video analysis.

numpy: A library that provides support for arrays and matrices, and includes functions for linear algebra, random number generation, and Fourier transforms.

sklearn.cluster.KMeans: A class from the scikit-learn machine learning library that implements the k-means clustering algorithm.

collections.Counter: A class from the collections module that provides a convenient way to count elements in a collection, such as a list or a dictionary.

tqdm: A library that provides a convenient way to display progress bars for long-running tasks.

With these libraries, you should be able to perform a wide range of data processing and analysis tasks.

In [44]:
import functools
import pathlib
import pickle
import os
import sqlite3
import json
import cv2
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
from tqdm import tqdm
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
import requests
from PIL import Image

# Settings base variables and paths
This code sets up the file structure and URL's for a project that uses data from an image dataset.


In [30]:
# Set the base folder path for the project
output_path = "../output"
images_path = os.path.join(output_path, "images")
metadata_path = os.path.join(output_path, "metadata")
config_path = os.path.join(output_path, "config")

list_of_paths = [output_path, images_path, metadata_path, config_path]

# Set the base URL for the dataset
metadata_extension = "sqlite"

yolo_cfg = "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"
yolo_weights = "https://pjreddie.com/media/files/yolov3.weights"
yolo_classes = "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

files_to_download = [
    (yolo_cfg, os.path.join(config_path, "yolov3.cfg")),
    (yolo_weights, os.path.join(config_path, "yolov3.weights")),
    (yolo_classes, os.path.join(config_path, "coco.names")),
]

# Create folder structure
The code creates the folder structure for the project. The folder structure is as follows:
- output
    - images
    - metadata
    - config

This method creates a folder with the given path if it doesn't already exist, It also outputs a message to inform the user if the folder was created or if it already exists.
This is useful for organizing and managing files in a project. By creating a folder to store data and resources, it keeps the working directory tidy and makes it easier to locate files. Additionally, by checking if the folder exists before creating it, it prevents the program from overwriting existing data or throwing an error.

In [31]:
def create_folder(path):
    """
    This function creates a folder at the specified path.
    If the folder already exists, it will print a message saying so.
    If there is an error creating the folder, it will print the error message.

    Parameters:
        :param path (str): The path of the folder to be created.

    Returns:
    None
    """
    try:
        # Use os.mkdir to create the folder at the specified path
        os.mkdir(path)
        print(f"Folder {path} created")
    except FileExistsError:
        # If the folder already exists, print a message saying so
        print(f"Folder {path} already exists")
    except Exception as e:
        # If there is an error creating the folder, print the error message
        print(f"Error creating folder {path}: {e}")

# Create the folder structure
This method initializes a list of folders by calling the create_folder method for each folder in the list.
The purpose of this method is to make sure that all necessary folders exist before the program continues its execution.
If a folder does not exist, the create_folder method will create it. If a folder already exists, the method will simply print a message indicating that the folder already exists. In case of any other error, the method will print the error message.

In [32]:
def init_folder(folder_names: list):
    for folder_name in folder_names:
        create_folder(folder_name)

In [33]:
init_folder(list_of_paths)

Folder ../output already exists
Folder ../output/images already exists
Folder ../output/metadata already exists
Folder ../output/config already exists


# Define methods for downloading the necessary files for yolo
The following code block is a method to download a file from a given URL and save it to a specified filename.
The method starts by creating a session (s = requests.Session()) and then mounting it to the URL (s.mount(url, requests.adapters.HTTPAdapter(max_retries=3))). This sets the maximum number of retries to 3 if the connection to the URL fails.
Then, the method makes a GET request to the URL (r = s.get(url, stream=True, allow_redirects=True)) and checks if it returns a successful response (r.raise_for_status()). If there was an HTTP error during the request, the error message is printed (print(f"HTTP error occurred while downloading dataset: {e}")).
The method also checks the file size specified in the response headers and assigns it to the variable file_size (file_size = int(r.headers.get(‘Content-Length’, 0))). If the file size is 0, a default value of “(Unknown total file size)” is assigned to the variable desc; otherwise, the variable desc is left empty.
Next, the method resolves the file path and creates a directory if it doesn’t already exist (path.parent.mkdir(parents=True, exist_ok=True)). The method then creates a tqdm progress bar to show the download progress (with tqdm.tqdm(total=file_size, unit=‘B’, unit_scale=True, desc=desc) as pbar:).
Finally, the method writes the contents of the file to disk in chunks (for chunk in r.iter_content(chunk_size=1024):), updating the progress bar for each chunk that is written to disk (pbar.update(len(chunk))). If an error occurred during the download, a message with the error is printed (print(f"Error occurred while downloading dataset: {e}")). The file path is returned when the method is finished.

In [34]:
def download(url, filename):
    """
    This download a file from a given URL and save it to a specified filename.

    Parameters:
        :param url (str): The URL of the file to be downloaded.
        :param filename (str): The filename to save the file as.

    Returns:
    path (str): The path of the downloaded file.
    """
    try:
        # Create a session object to persist the state of connection
        s = requests.Session()
        # Retry connecting to the URL up to 3 times
        s.mount(url, requests.adapters.HTTPAdapter(max_retries=3))
        # Send a GET request to the URL to start the download
        r = s.get(url, stream=True, allow_redirects=True)
        # Raise an error if the response is not 200 OK
        r.raise_for_status()
        # Get the file size from the Content-Length header, default to 0 if not present
        file_size = int(r.headers.get('Content-Length', 0))
        # Get the absolute path to the target file
        path = pathlib.Path(filename).expanduser().resolve()
        # Create parent directories if they don't exist
        path.parent.mkdir(parents=True, exist_ok=True)
        # Set the description to display while downloading, "(Unknown total file size)" if file size is 0
        desc = "(Unknown total file size)" if file_size == 0 else ""
        # Enable decoding the response content
        r.raw.read = functools.partial(r.raw.read, decode_content=True)
        # Use tqdm to display the download progress
        with tqdm(total=file_size, unit='B', unit_scale=True, desc=desc) as pbar:
            # Open the target file in binary write mode
            with path.open("wb") as f:
                # Write each chunk of data from the response to the file
                for chunk in r.iter_content(chunk_size=1024):
                    f.write(chunk)
                    pbar.update(len(chunk))
        # Return the path to the downloaded file
        return path
    # Handle HTTP error if the response is not 200 OK
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred while downloading dataset: {e}")
    # Handle any other exceptions that might occur while downloading the file
    except Exception as e:
        print(f"Error occurred while downloading dataset: {e}")

# Download the yolov3 files if they don't already exist
This code block downloads the yolov3 files if they don't already exist. The files_to_download list contains tuples of the URL and the filename to save the file as. The method iterates through the list and checks if the file already exists. If the file doesn't exist, the download method is called to download the file. If the file already exists, a message is printed indicating that the file already exists.

In [35]:
for url, filename in files_to_download:
    # check if the file already exists
    if not pathlib.Path(filename).exists():
        # download the file
        download(url, filename)

print("All files are downloaded")

All files are downloaded


# Define methods to get all the image paths
The get_all_images method is used to retrieve all images present in the specified image path. It uses the os.walk function to traverse through all subdirectories within the image path and collects the file names that end with either '.png' or '.jpg' extensions. The full path of each image is then generated by joining the root directory and the file name. The method returns a list of all images' full paths. In case of any error, an error message is printed and an empty list is returned.

In [36]:
def get_all_images(path):
    """Get all images from the given path.

    Args:
    param: image_path (str): path to the directory containing the images.

    Returns:
    - list: a list of full path to all the images with png or jpg extensions.
    - empty list: an empty list if an error occurred while fetching images.
    """
    try:
        # use os.walk to traverse all the subdirectories and get all images
        return [os.path.join(root, name)
                for root, dirs, files in os.walk(path)
                for name in files
                if name.endswith((".png", ".jpg"))]
    except Exception as e:
        # return an empty list and log the error message if an error occurred
        print(f"An error occurred while fetching images: {e}")
        return []

# Yolo for tagging

This part of contains code for object detection using YOLO (You Only Look Once) deep learning architecture. The code is implemented using OpenCV and Python.

The code contains three functions get_labels, get_colors, and detect that work together to perform the object detection task.

get_labels takes the path to a label file as input and returns the labels in the file. If the label file does not exist, it prints a message and returns an empty list.

get_colors generates random RGB color codes for each label in the input list.

detect is the main function that performs the object detection task. It takes several parameters such as the path to the image, the paths to the pre-trained model files (weights, configuration, and labels), and several hyperparameters (confidence, score threshold, and IOU threshold).

The function first loads the labels using the get_labels function and generates random colors for each label using the get_colors function. Then, it loads the pre-trained YOLO model using OpenCV's cv2.dnn.readNetFromDarknet method.

The input image is then passed through the network, and the output is obtained by calling the net.forward method. The output is then processed to get the class IDs, confidences, and bounding box coordinates for each detected object.

Finally, the bounding boxes are drawn on the image, and the labels and confidences scores are displayed on the image. The output image is saved to the specified output folder.

In summary, this code provides an implementation of the YOLO object detection algorithm using OpenCV and Python.

In [37]:

def get_labels(label_path):
    # check if the file exists
    if not pathlib.Path(label_path).exists():
        print(f"Label file {label_path} does not exist")
        return []
    return open(label_path).read().strip().split("\n")

def get_colors(labels):
    return np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

def detect(
        path_name,
        output_folder="../output/images_labelized/",
        label_path="../output/config/coco.names",
        weights_path="../output/config/yolov3.weights",
        config_path="../output/config/yolov3.cfg",
        CONFIDENCE=0.5,
        SCORE_THRESHOLD=0.5,
        IOU_THRESHOLD=0.5
):
    labels = get_labels(label_path)
    colors = get_colors(labels)

    # load the COCO class labels our YOLO model was trained on
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

    image = cv2.imread(path_name)
    file_name = os.path.basename(path_name)
    filename, ext = file_name.split(".")

    h, w = image.shape[:2]
    # create 4D blob
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # sets the blob as the input of the network
    net.setInput(blob)
    # get all the layer names
    ln = net.getLayerNames()
    print(ln)
    try:
        ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    except IndexError:
        # in case getUnconnectedOutLayers() returns 1D array when CUDA isn't available
        ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
    # feed forward (inference) and get the network output
    # measure how much it took in seconds
    layer_outputs = net.forward(ln)

    font_scale = 1
    thickness = 1
    boxes, confidences, class_ids = [], [], []
    text_labels = []
    # loop over each of the layer outputs
    for output in layer_outputs:
        # loop over each of the object detections
        for detection in output:
            # extract the class id (label) and confidence (as a probability) of
            # the current object detection
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # discard out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > CONFIDENCE:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

        # loop over the indexes we are keeping
        for i in range(len(boxes)):
            text_labels.append(labels[class_ids[i]])
            # extract the bounding box coordinates
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]
            # draw a bounding box rectangle and label on the image
            color = [int(c) for c in colors[class_ids[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
            text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
            # calculate text width & height to draw the transparent boxes as background of the text
            (text_width, text_height) = \
            cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
            text_offset_x = x
            text_offset_y = y - 5
            box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
            overlay = image.copy()
            cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
            # add opacity (transparency to the box)
            image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
            # now put the text (label: confidence %)
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=font_scale, color=(0, 0, 0), thickness=thickness)

    cv2.imwrite(os.path.join(output_folder, filename + "." + ext), image)

    return text_labels


In [38]:
!pip install transformers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [45]:
def detect(image):
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101")
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # convert outputs (bounding boxes and class logits) to COCO API
    # let's only keep detections with score > 0.9
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
    labels = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        labels.append(model.config.id2label[label.item()])
        print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
        )
    return labels


# Save  metadata

The function save_metadata allows you to save metadata information of an image in either pickle, json, or sqlite format. The function takes four parameters: metadata, img_name, metadata_path, and save_format.

metadata is a dictionary that contains the metadata information of an image. img_name is a string that represents the file name of the image. metadata_path is a string that specifies the path to the directory where the metadata will be saved. save_format is an optional parameter that specifies the format in which the metadata will be saved. The default value is pickle.

The function saves the metadata in the specified format. If save_format is set to pickle, the metadata is saved in the pickle format. If save_format is set to json, the metadata is saved in the json format. If save_format is set to sqlite, the metadata is saved in the sqlite database.

If an error occurs while saving the metadata, the function will print an error message indicating the image name and the error that occurred.

The function does not return any value.

In [46]:
def save_metadata(metadata, img_name, metadata_path, save_format='pickle'):
    """
    This function saves the metadata information of an image in either pickle or json format.
    Parameters:
    metadata (dict): The metadata information of an image.
    img_name (str): The file name of the image.
    metadata_path (str): The path to the directory where the metadata will be saved.
    save_format (str): The format in which the metadata will be saved. The default is 'pickle'.

    Returns:
    None
    """
    try:
        if save_format == 'pickle':
            # save the metadata in pickle format
            with open(os.path.join(metadata_path, os.path.splitext(os.path.basename(img_name))[0] + '.pickle'),
                      'wb') as f:
                pickle.dump(metadata, f)
        elif save_format == 'json':
            # save the metadata in json format
            with open(os.path.join(metadata_path, os.path.splitext(os.path.basename(img_name))[0] + '.json'), 'w') as f:
                json.dump(metadata, f)
        elif save_format == 'sqlite':
            # Get only the file name of the image
            img_name = os.path.basename(img_name)
            # Open a connection to the database
            conn = sqlite3.connect(os.path.join(metadata_path, 'metadata.db'))
            # Create a cursor
            c = conn.cursor()
            # Create a table if it doesn't exist : filename, key, value
            c.execute('''CREATE TABLE IF NOT EXISTS metadata (filename text, key text, value text)''')
            # Insert the metadata into the table
            for key, value in metadata.items():
                # Convert key, value to string
                key = str(key)
                value = str(value)
                # Check if the key is already in the table
                c.execute("SELECT * FROM metadata WHERE filename=? AND key=?", (img_name, key))
                # If the key is already in the table, update the value
                if c.fetchone():
                    c.execute("UPDATE metadata SET value=? WHERE filename=? AND key=?", (value, img_name, key))
                    # Commit the changes
                    conn.commit()
                # If the key is not in the table, insert the key, value pair
                else:
                    c.execute("INSERT INTO metadata VALUES (?, ?, ?)", (img_name, key, value))
                    # Commit the changes
                    conn.commit()
            # Close the connection
            conn.close()
        else:
            raise ValueError("Invalid save format")
    except Exception as e:
        # print an error message if an error occurs
        print(f"An error occurred while saving metadata for {img_name}: {str(e)}")

# Read SQLite metadata

The read_sqlite method is a function used to read metadata information from a SQLite database. The method takes two parameters: metadata_path, the path to the directory where the metadata is saved, and filename, the name of the file for which the metadata is to be retrieved.

The method starts by connecting to the SQLite database located at metadata_path/metadata.db using the sqlite3.connect method. A cursor is then created to allow interaction with the database. If the metadata table doesn't exist in the database, it is created.

The metadata for the specified filename is retrieved from the database by executing a SQL query that selects all rows where the filename column is equal to the filename parameter. The retrieved metadata is stored in a dictionary, where the keys are taken from the key column and the values from the value column.

Finally, the database connection is closed and the metadata dictionary is returned as the result of the function.

In [47]:
def read_sqlite(metadata_path, filename):
    # Open a connection to the database
    conn = sqlite3.connect(os.path.join(metadata_path, 'metadata.db'))
    # Create a cursor
    c = conn.cursor()
    # Create a table if it doesn't exist : filename, key, value
    c.execute('''CREATE TABLE IF NOT EXISTS metadata (filename text, key text, value text)''')
    # Insert the metadata into the table
    c.execute("SELECT * FROM metadata WHERE filename=?", (filename,))
    # If the key is already in the table, update the value
    metadata = {}
    for row in c.fetchall():
        metadata[row[1]] = row[2]
    # Close the connection
    conn.close()
    return metadata

# Set tags in metadata
This function "update_tags" is used to run the YOLOv3 algorithm on a set of images, update the metadata of each image with the detected labels (tags) and save the updated metadata.

The function takes 3 parameters:

images: a list of file paths for the images that need to be processed.
metadata_path: a file path to the directory where the metadata files are stored.
save_format: the format of the metadata files. Can be either 'pickle' or 'sqlite'.
The function uses the tqdm library to display a progress bar for the image processing. For each image, the function tries to retrieve its metadata based on the save_format. If the metadata file format is 'sqlite', the function calls the read_sqlite function to retrieve the metadata. If the metadata file format is 'pickle', the function reads the metadata file directly.

If the metadata already contains a "tags" key, it means that the image has already been processed and its metadata has been updated with the labels, so the function skips that image.

The function then calls the detect function to run the YOLOv3 algorithm on the image and retrieve the labels (tags). The labels are added to the metadata under the "tags" key.

Finally, the function calls the save_metadata function to save the updated metadata. If an error occurs while processing an image (e.g. the metadata file is not found), the function prints an error message and continues processing the next image.

In [48]:
def update_tags(images, metadata_path, save_format='pickle'):
    # Run the YOLOv3 algorithm on each image
    # display progress bar in the first thread only
    for image in tqdm(images, desc="Updating tags"):
        # read pickle file from ../output/metadata/file_name.pkl
        file_name = os.path.basename(image)
        file_name, ext = file_name.split(".")
        try:
            if save_format == 'sqlite':
                metadata = read_sqlite(metadata_path, file_name + "." + ext)
            else:
                with open(os.path.join("../output/metadata", file_name + "." + metadata_extension), "rb") as f:
                    if metadata_extension == "json":
                        metadata = json.load(f)
                    else:
                        metadata = pickle.load(f)

            if "tags" in metadata:
                continue

            image = Image.open(image)
            labels = detect(image)

            # Remove duplicates from labels
            labels = list(set(labels))
            # add labels to metadata
            metadata["tags"] = labels

            save_metadata(metadata, file_name + "." + ext, metadata_path, save_format)
        except FileNotFoundError:
            print("File not found: ", file_name)
            continue
        except Exception as e:
            print("Error: ", e)
            continue

In [49]:
# Get the list of images
images = os.listdir(images_path)
images = [os.path.join(images_path, image) for image in images]

update_tags(images, metadata_path, save_format='sqlite')

Updating tags:  17%|█▋        | 785/4491 [00:07<00:36, 101.38it/s]/Users/yannisteissier/Documents/bigdata/projet/venv/lib/python3.9/site-packages/transformers/models/detr/image_processing_detr.py:776: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(
Updating tags:  18%|█▊        | 806/4491 [00:10<03:15, 18.82it/s] 

Error:  Unsupported number of image dimensions: 2


Updating tags:  30%|██▉       | 1333/4491 [00:16<00:42, 74.37it/s]

Error:  cannot identify image file '../output/images/image_sEDzxW4NhL4.jpg'


Updating tags:  36%|███▋      | 1632/4491 [00:21<02:35, 18.38it/s]

...

Updating tags: 100%|█████████▉| 4484/4491 [2:14:44<00:27,  3.91s/it]

Detected person with confidence 0.999 at location [19.59, 806.03, 1539.85, 2962.07]


Updating tags: 100%|█████████▉| 4486/4491 [2:14:51<00:18,  3.75s/it]

Detected potted plant with confidence 0.978 at location [32.93, 113.54, 1907.49, 1899.22]


Updating tags: 100%|██████████| 4491/4491 [2:15:11<00:00,  1.81s/it]


### Now, find dominant colors in the images
The functions rgb_to_hex and find_dominant_colors are used to find the dominant colors in an image.

The function rgb_to_hex takes in an RGB array with 3 values, and returns the hexadecimal representation of the color. This can be useful for formatting colors in a standardized way, as hexadecimal codes are widely used in web development and other applications.

The function find_dominant_colors takes in an image and optional parameters k and image_processing_size. The k parameter specifies the number of dominant colors to return, with a default value of 4. The image_processing_size parameter allows you to resize the image to a smaller size, to speed up the processing, if desired.

The image is first converted from BGR to RGB, and then reshaped into a list of pixels. The KMeans algorithm is used to cluster the pixels into k clusters, and the most popular clusters are identified. The color values for each of the k clusters are converted to hexadecimal representation and returned as a list, along with the percentage of the image covered by each color.

In [50]:
import numpy as np
from collections import Counter
from sklearn.cluster import KMeans
from joblib import Parallel, delayed
import cv2

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % (int(rgb[0]), int(rgb[1]), int(rgb[2]))

def find_dominant_colors(image, k=4, image_processing_size=None):
    if image_processing_size is not None:
        image = cv2.resize(image, image_processing_size, interpolation=cv2.INTER_LINEAR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape(-1, 3)
    clt = KMeans(n_clusters=k, n_init=10)
    labels = clt.fit_predict(image)
    label_counts = Counter(labels)
    dominant_colors = (clt.cluster_centers_[i] for i in label_counts.keys())
    dominant_colors = [rgb_to_hex(color) for color in dominant_colors]
    percent = [int((label_counts[i] / len(labels)) * 100) for i in label_counts.keys()]
    dominant_colors = list(zip(dominant_colors, percent))
    return dominant_colors


The following code block is used to process images and find their dominant colors. The code first retrieves all the images present in the folder specified by the images_path variable. Then, it iterates over each image, reads the metadata associated with the image and finds its dominant color if it hasn't been calculated already.

For each image, the code first reads the image using OpenCV's cv2.imread() function and stores the result in the img variable. The code then reads the metadata of the image. The type of metadata file (e.g. .json, .pkl, .sqlite) is specified by the metadata_extension variable. Based on the file extension, the code reads the metadata using either read_sqlite(), json.load(), or pickle.load() functions. If the metadata file is not found, the code continues to the next iteration of the loop, but if there is an error, it prints the error message and continues to the next iteration.

If the metadata does not contain information about the dominant color of the image, the code calculates the dominant color by calling the find_dominant_colors() function. The result of the find_dominant_colors() function is then added to the metadata under the key "dominant_color". Finally, the updated metadata is saved using the save_metadata() function, which saves the metadata to the specified location using the specified file format (metadata_extension).

In [1]:
images = get_all_images(images_path)

# For each image in the folder "../images_labelized"
for image in tqdm(images, desc="Finding dominant colors"):
    img = cv2.imread(image)
    # Save the dominant color
    # First read metadata from the file
    file_name = os.path.basename(image)
    file_name, ext = file_name.split(".")
    try:
        if metadata_extension == 'sqlite':
            metadata = read_sqlite("../output/metadata", file_name + "." + ext)
        else:
            with open(os.path.join("../output/metadata", file_name + "." + metadata_extension), "rb") as f:
                if metadata_extension == "json":
                    metadata = json.load(f)
                else:
                    metadata = pickle.load(f)

        if "dominant_color" in metadata:
            continue

        dominant_color = find_dominant_colors(img)

        # Add the dominant color to the metadata
        metadata["dominant_color"] = dominant_color
        # Save the metadata
        save_metadata(metadata, file_name + "." + ext, "../output/metadata", metadata_extension)
    except FileNotFoundError:
        print("File not found: ", file_name)
        continue
    except Exception as e:
        print("Error: ", e)
        continue


NameError: name 'get_all_images' is not defined

In [2]:
# Zip all files in output/images
# and output/metadata into output.zip
import shutil
shutil.make_archive("../output", "zip", "../output")

'/Users/yannisteissier/Documents/bigdata/projet/output.zip'

'/Users/yannisteissier/Documents/bigdata/projet/output.zip'